In [39]:
!pip install gspread pandas oauth2client

In [42]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import json
import re
import os



# Ưu tiên lấy từ biến môi trường
if "SERVICE_ACCOUNT_JSON" in os.environ:
    service_account_info = json.loads(os.environ["SERVICE_ACCOUNT_JSON"])
else:
    # Fallback: đọc từ file service_account.json (bạn phải upload lên Colab hoặc để sẵn trong project)
    with open("service_account.json") as f:
        service_account_info = json.load(f)
# --- 1. Authenticate ---
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scope)
client = gspread.authorize(creds)

# Mở Google Sheet
spreadsheet = client.open_by_url(
    "https://docs.google.com/spreadsheets/d/1kzBnsanuUbyez4tlaqydXQYfKPHTWg0U5X5-KwpcF6w/edit?gid=1180872507#gid=1180872507"
)

# --- 3. Define team regions ---
hn_teams = ['Panther', 'Lion', 'News', "Trộm vía"]
hcm_teams = ['Sandy', 'Victory', 'Mineral', 'New HCM', 'Phoenix', 'Diamond']

all_data = []

# --- 4. Quét qua tất cả sheet ---
for ws in spreadsheet.worksheets():
    if re.match(r"^T\d+$", ws.title):  # chỉ lấy sheet có tên T + số
        values = ws.get_all_values()
        if len(values) < 2:
            continue  # skip sheet trống

        # Lấy dòng đầu tiên làm header, loại bỏ header trống
        header = [h if h != "" else f"col_{i}" for i, h in enumerate(values[0])]
        df = pd.DataFrame(values[1:], columns=header)

        # Thêm cột Month
        df["Month"] = int(ws.title[1:])

        # Thêm cột Vùng
        if "Tên team" in df.columns:
            df["Vùng"] = df["Tên team"].apply(
                lambda x: "HN" if x in hn_teams else ("HCM" if x in hcm_teams else "Other")
            )

        all_data.append(df)

# --- 5. Gộp tất cả sheet ---
df_all = pd.concat(all_data, ignore_index=True)

# --- 6. Lọc Leader ---
if "Team" in df_all.columns:
    leaders_df = df_all[df_all["Team"] == "Leader"]
else:
    leaders_df = pd.DataFrame()  # tránh lỗi nếu thiếu cột Team

# --- 7. Thêm KPI Vùng ---
if "Vùng" in leaders_df.columns:
    leaders_df["KPI Vùng"] = leaders_df["Vùng"].map({
        "HCM": 1943993473,
        "HN": 700000000
    })
# --- 8. Chỉ lấy cột cần thiết ---
cols_needed = [c for c in ["Tên team", "Month", "Vùng", " KPI ", "KPI Vùng"] if c in leaders_df.columns]
leaders_df = leaders_df[cols_needed]



# --- 9. Loại bỏ dòng có KPI TEAM trống ---
# Tìm đúng cột KPI TEAM
kpi_col = None
for c in ["KPI", " KPI ", "KPI TEAM"]:
    if c in leaders_df.columns:
        kpi_col = c
        break

if kpi_col:
    leaders_df = leaders_df.dropna(subset=[kpi_col])  # bỏ NaN
    leaders_df = leaders_df[leaders_df[kpi_col].astype(str).str.strip() != ""]  # bỏ chuỗi rỗng


leaders_df[kpi_col] = (
      leaders_df[kpi_col]
      .astype(str)
      .str.replace(".", "", regex=False)  # bỏ dấu chấm ngăn cách
      )
leaders_df[kpi_col] = pd.to_numeric(leaders_df[kpi_col], errors="coerce")


if kpi_col:
    leaders_df = leaders_df.loc[
        leaders_df.groupby(["Tên team", "Month"])[kpi_col].idxmax()
    ].reset_index(drop=True)



FileNotFoundError: [Errno 2] No such file or directory: 'service_account.json'

In [38]:
# Xuất DataFrame df sang Google Sheets
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1C3YMR7tsGRldYQiwCVHUAilaTwMwTypoBO4fsFdlh2c/edit?gid=283772782#gid=283772782")
worksheet = spreadsheet.worksheet("KPI TEAM")      # tab đầu tiên
worksheet.clear()                                # xoá dữ liệu cũ
worksheet.update([leaders_df.columns.values.tolist()] + leaders_df.values.tolist())

print("✅ Đã ghi dữ liệu thành công lên Google Sheets!")


✅ Đã ghi dữ liệu thành công lên Google Sheets!
